# Simple error detection for 1 bit flips in a message

Hamming codes are methods for error-detection and error-correction of a transmitted piece of information. All information(text, images, audio, video, etc.) is ultimately processed by the computer as a stream of 1's and 0's, and it's common for errors to creep in when transmitting a piece of data/information across a noisy channel. Thus, methods like Hamming Codes were invented to counteract this. This is why a scratched DVD, if not completely damaged, can still be played with decent quality. Although Hamming codes, first invented by Richard Hamming in the early 50s, are now an outdated method, it has historical significance in that it served as the foundation for modern error correction methods modern systems use. I found Hamming interesting enough to want to try to implement them in code in order to understand it better. 
This notebook isn't meant to be an explanation or tutorial of Hamming codes, so if you're interested in learning more about them, 3Blue1Brown gives a great overview of it: https://www.youtube.com/watch?v=X8jsijhllIA . 


In [221]:
import numpy as np
from functools import reduce
import math

In [268]:
class Message:
  PARITY_IDX = [1,2,4,8,16,32] # indices in the message block reserved for parity

  def __init__(self, msg = None, msg_len=16):
    self.message = msg
    if len(self.message) != msg_len:    
      # by default set 0th bit to 0. As of right now, because this is a (15,11) Hamming Code implementation, the zeroth bit is essentially ignored.
      self.message = np.insert(self.message,0,0)
    self.msg_len = len(self.message)
    self.dim = int(math.sqrt(len(self.message)))
    self.block = self.message.reshape(self.dim, self.dim) # the message block in matrix form (for easier visualization) 
    self.one_idx = [x for x,y in enumerate(self.message) if y] # list of(index, bit-value) tuples where the bit-value is one


  def set_parity(self):
    """
    Flips the parity bits that need to be flipped to reach parity.
    """
    pc = self.parity_check()
    if pc == 0:
      return "Already in parity!"
    idx = [self.PARITY_IDX[i] for i in pc] # indexes of parity bits that need to be flipped
    self.flip_bits(idx)
    self.one_idx = [x for x,y in enumerate(self.message) if y]


  def parity_check(self):
    """
    Performs parity check. If message in full parity, return True. 
    Else returns index of parity bits which need to be flipped.
    """
    xor_msg = xor(self.get_ones())
    if xor_msg == 0:
      return xor_msg
    else:
      xor_msg = bin(xor_msg)[:1:-1]
      return [x for x,y in enumerate(xor_msg) if y == '1']  
  

  def ready_to_send(self):
    """
    Returns if the message block is ready to be sent to Receiver
    """
    if self.parity_check() == 0:
      return True
    else:
      return "Message is not in parity"


  def flip_bits(self, idx):
    """
    Takes in a list of indexes and flips the bit located at each index
    """
    for i in idx:
      self.message[i] = 1- self.message[i]
    self.one_idx = self.get_ones() # update the 1 indices


  def get_ones(self):
    """Returns a list with the indexes of slots with a 1"""
    ones = [x for x,y in enumerate(self.message) if y]
    self.one_idx = ones
    return ones


  def __repr__(self):
      return str(self.message)
  
  def __eq__(self, other):
    return np.array_equal(self.message, other.message)


In [159]:
class Receiver:
  def __init__(self):
    self.received_msg = None  
    #self.test = Message()


  def receive_message(self, message: Message):
    """
    Takes in a message sent from the Sender class
    """
    self.received_msg = message
  

  def scan(self, msg: Message):
    """
    Performs parity check on the received message and returns the index of the error, if there are any. 
    """
    error = xor(msg.get_ones())
    if error == 0:
      return "No errors found"
    return f"Error found at index {error}"


  def correct_error(self, msg: Message):
    error = xor(msg.get_ones())
    if error == 0:
      return "No errors detected"
    msg.flip_bits([error])
    self.received_message = msg.block
    return msg
  
  

In [4]:
class Sender:
  def __init__(self):
    pass

  def send_message(self, msg: Message, receiver: Receiver, distort=False):
    """
    Sends the message to Receiver. If distort is set to true, a 1-bit error will be introduced into the message
    the receiver receives. 
    """
    if not isinstance(msg, Message):
      msg = Message(msg)
    self.sent_msgs = msg
    if distort is True:
      msg = msg.distort_msg(random=True)
  
    receiver.receive_message(msg)
  
  def set_parity(self, msg:Message):
    msg.set_parity()
    


In [266]:
def distort_msg(msg: Message, idx = 5, random=False) -> Message:
  """
  Intentionally introduces a 1-bit error in the message by flipping the bit in the message's idx-th index. 
  If random set to True, a randomly chosen (except for the zero-th) bit will be flipped. 
  """
  if random is True:
    idx = np.random.randint(1, msg.msg_len)

  error_msg = Message(np.copy(msg.message), msg_len=len(msg.message))
  # error_msg.message[idx] = 1 - error_msg.message[idx]
  error_msg.flip_bits([idx])
  return error_msg
  

def xor(lst):
  """returns the xor of all elements of the list x"""
  return reduce(lambda x,y : x^y, lst)




# Let's test it out!

In [231]:
# Initialize Bob, the Sender of the message, and Alice, the receiver of the message

Bob=Sender()
Alice=Receiver()

In [234]:
# Generate random 16-bit message for Bob to send to Alice (in practice, this could be anything that can be represented as a bitstring ie text, image, audio, etc.)
# Technically, only 11 of the 16 bits are meant for the actual message, and the other 5 bits are reserved for parity. 
# But for the purposes of this implementation, we're going to treat the entire 16-bit block as the message Bob intends to send to Alice.

bob_msg = np.random.randint(0,2,15)
bob_msg = Message(bob_msg)

# View the message
bob_msg

[0 1 1 1 0 1 0 0 0 0 1 0 1 1 0 1]

In [235]:
# Set the parity of the message to prepare it for transmission to Alice
Bob.set_parity(bob_msg)

# View the message block in parity
bob_msg

[0 0 1 1 0 1 0 0 0 0 1 0 1 1 0 1]

In [236]:
# Confirm the message is in parity and ready to send
bob_msg.ready_to_send()

True

In [237]:
# Oh no!!! As Bob is sending the message to Alice, a random one bit error accidentally creeps in. 
# We don't yet know which bit was distorted in the message.
error_msg = distort_msg(bob_msg, random=True)
Bob.send_message(error_msg, Alice)


In [238]:
# View the message with the one bit error
error_msg

[0 0 1 1 0 1 0 0 0 1 1 0 1 1 0 1]

In [239]:
print(f"Original message:   {bob_msg}")
print(f"Message with error: {error_msg}")

Original message:   [0 0 1 1 0 1 0 0 0 0 1 0 1 1 0 1]
Message with error: [0 0 1 1 0 1 0 0 0 1 1 0 1 1 0 1]


In [240]:
# Confirm the message with the error is no longer the same as the original message Bob intended to send to Alice
error_msg == bob_msg

False

In [241]:
# confirm Alice received message
Alice.received_msg

[0 0 1 1 0 1 0 0 0 1 1 0 1 1 0 1]

In [242]:
error_msg == Alice.received_msg

True

In [243]:
# Alice scans the message to determine whether an error exists in the message she received from Bob.
# If an error is detected, she can identify the location of the error (the error's index in the message)
# Keep in mind that throughout this whole process, Alice never sees the "correct" message Bob originally meant to send. 
# She can deduce if there was an error and where it is just by looking at the erroneous message she received 

Alice.scan(error_msg)

'Error found at index 9'

In [244]:
# Once Alice identifies the location of the error, she corrects it by flipping the bit
corrected_msg = Alice.correct_error(error_msg)
corrected_msg

[0 0 1 1 0 1 0 0 0 0 1 0 1 1 0 1]

In [245]:
print(f"Original message:         {bob_msg}")
print(f"Message after correction: {corrected_msg}")

Original message:         [0 0 1 1 0 1 0 0 0 0 1 0 1 1 0 1]
Message after correction: [0 0 1 1 0 1 0 0 0 0 1 0 1 1 0 1]


In [246]:
# Verify if the corrected message is the same as Bob's original message
corrected_msg == bob_msg

True

## AND THUS (15,11) HUFFMAN ENCODING WORKS   🔥🔥🔥  	

In [251]:
# Lets try on a 64 bit message

bob_msg = np.random.randint(0,2,63)
bob_msg = Message(bob_msg)

bob_msg


[0 1 0 0 1 1 1 1 1 1 0 0 0 0 1 0 0 1 0 1 1 1 0 0 0 1 1 0 1 0 1 0 0 1 0 0 0
 0 0 0 1 1 1 0 0 1 0 0 1 1 0 1 1 1 0 1 0 1 0 1 1 1 0 1]

In [252]:
len(bob_msg.message)

64

In [253]:
Bob.set_parity(bob_msg)
bob_msg.ready_to_send()


True

In [258]:
error_msg = distort_msg(bob_msg)
Bob.send_message(error_msg, Alice)

error_msg == bob_msg

False

In [261]:
error_msg == Alice.received_msg

True

In [262]:
Alice.scan(error_msg)

'Error found at index 2'

In [263]:
corrected_msg = Alice.correct_error(error_msg)
corrected_msg

[0 0 1 0 1 1 1 1 1 1 0 0 0 0 1 0 1 1 0 1 1 1 0 0 0 1 1 0 1 0 1 0 0 1 0 0 0
 0 0 0 1 1 1 0 0 1 0 0 1 1 0 1 1 1 0 1 0 1 0 1 1 1 0 1]

In [264]:
corrected_msg == bob_msg

True